In [95]:
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline


import os

# Подготовим данные

In [96]:
bank = pd.read_csv('bank/bank-full.csv', sep=';', decimal=",")
bank.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [97]:
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()

bank['job'] = labelencoder.fit_transform(bank['job'])
bank['marital'] = labelencoder.fit_transform(bank['marital'])
bank['education'] = labelencoder.fit_transform(bank['education'])
bank['default'] = labelencoder.fit_transform(bank['default'])
bank['housing'] = labelencoder.fit_transform(bank['housing'])
bank['loan'] = labelencoder.fit_transform(bank['loan'])
bank['contact'] = labelencoder.fit_transform(bank['contact'])
bank['month'] = labelencoder.fit_transform(bank['month'])
bank['poutcome'] = labelencoder.fit_transform(bank['poutcome'])
bank['y'] = labelencoder.fit_transform(bank['y'])

bank.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,4,1,2,0,2143,1,0,2,5,8,261,1,-1,0,3,0
1,44,9,2,1,0,29,1,0,2,5,8,151,1,-1,0,3,0
2,33,2,1,1,0,2,1,1,2,5,8,76,1,-1,0,3,0
3,47,1,1,3,0,1506,1,0,2,5,8,92,1,-1,0,3,0
4,33,11,2,3,0,1,0,0,2,5,8,198,1,-1,0,3,0


Разделим на предикторы и отклики

In [98]:
X = bank.iloc[:, :-1].values  
y = bank.iloc[:, 16].values 

Разделим выборку на обучающую и тестовую (0.33 от всей выборки)

In [99]:
from sklearn.model_selection import train_test_split  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state = 1234)  

# Строим модель деревья классификации

In [100]:
from sklearn.tree import DecisionTreeClassifier

Инициализируем и обучаем модель (сначала подберем параметры вручную)

In [101]:
model = DecisionTreeClassifier(random_state=42,
                               # можно выбрать 'entropy', если модель построится некачественной даже после 
                               # изменения других параметров
                               criterion='gini',
                               # максимальная глубина дерева (пока выберем 6, кажется, что 
                               # не слишком мало и не слишком много)
                               max_depth=6,
                               # максимальное число конечных узлов (не задаем ограничение, потому что 
                               # глубину дерева задачи уже, этого хватит для ограничения)
                               max_leaf_nodes=None,
                               # минимальное число элементов в узле для разбиения (пока пустьт будет 10)
                               min_samples_split=10,
                               # минимальное число элементов в потомке (пока пусть будет 8)
                               min_samples_leaf=8,
                               # минимальное значение уменьшения загрязнения (разработчики советуют брать 1/1000)
                               min_impurity_decrease=0.001,
                               class_weight=None,
                               # Используем, если задаем веса классов
                               min_weight_fraction_leaf=0,                              
                               # какое расщепление выбирать? ('best' или 'random' - второе не очень понятно, зачем
                               # лучше выбирать лучшее)
                               splitter='best',
                               # Перебирать все переменные
                               max_features = None)

model.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=6, min_impurity_decrease=0.001,
                       min_samples_leaf=8, min_samples_split=10,
                       min_weight_fraction_leaf=0, random_state=42)

Распознаем, используя дерево CART

In [102]:
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

In [103]:
#  Распознаем вероятности принадлежать классам.
y_pred_train2 = model.predict_proba(X_train)
y_pred_test2 = model.predict_proba(X_test)

Посмотрим важность каждого параметра

In [104]:
pd.DataFrame({'feature': bank.iloc[:, :-1].columns,
              'importance': model.feature_importances_}).sort_values('importance', ascending=False)

,feature,importance
11,duration,0.549242
15,poutcome,0.168027
10,month,0.100591
13,pdays,0.072473
0,age,0.043307
8,contact,0.042258
6,housing,0.024102
1,job,0.000000
2,marital,0.000000
3,education,0.000000


Видно, что больше половины параметров не являются значимыми

# Оценим построенную модель

In [105]:
from sklearn import metrics

Построим матрицы ошибок на тестовой и обучающей выборках

In [106]:
conf_mat = metrics.confusion_matrix(y_test, y_pred_test)
conf_mat = pd.DataFrame(conf_mat, index=model.classes_, columns=model.classes_)
conf_mat

,0,1
0,12780,398
1,1114,628


In [107]:
conf_mat = metrics.confusion_matrix(y_train, y_pred_train)
conf_mat = pd.DataFrame(conf_mat, index=model.classes_, columns=model.classes_)
conf_mat

,0,1
0,25958,786
1,2286,1261


Вывод: пропорциональность сохраняется: доля неправильно определенных классов в обучающей выборке (786, 2286) примерно в 2 раза больше, чем в тестовой (398, 1114), как и количества наблюдений в этих выборках

In [108]:
from sklearn.metrics import accuracy_score
print("Accuracy on train set is ", accuracy_score(y_train, y_pred_train))
print("Accuracy on test set is ", accuracy_score(y_test, y_pred_test))

Accuracy on train set is  0.8985837377438843
Accuracy on test set is  0.8986595174262735


Вывод: точность на тестовой и обучающей выборках сильно не отличаются, можно сказать, что модель не переобучилась.

# Воспользуемся поиском гиперпараметров на сетке GridSearch

In [109]:
from sklearn.model_selection import GridSearchCV

Рассмотрим параметры чуть меньше и чуть больше тех, которые были взяты

In [110]:
params_set = {
'max_depth': [4, 6, 7],
'min_samples_split': [8, 10, 12],
'min_samples_leaf': [4, 8, 10]
}

Инициализируем модель, в которой будем менять параметры из сетки

In [111]:
model_2 = DecisionTreeClassifier(random_state=42,
                               criterion='gini',
                               # максимальная глубина дерева (будет меняться)
                               max_depth=6,
                               max_leaf_nodes=None,
                               min_samples_split=10,
                               min_samples_leaf=8,
                               min_impurity_decrease=0.001,
                               class_weight=None,
                               min_weight_fraction_leaf=0,                              
                               splitter='best',
                               max_features = None)

In [112]:
grid_CV_1 = GridSearchCV(estimator=model_2,
       param_grid=params_set,
       scoring='accuracy',
       cv=5,
       n_jobs=-1)

grid_CV_1.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=DecisionTreeClassifier(max_depth=6,
                                              min_impurity_decrease=0.001,
                                              min_samples_leaf=8,
                                              min_samples_split=10,
                                              min_weight_fraction_leaf=0,
                                              random_state=42),
             n_jobs=-1,
             param_grid={'max_depth': [4, 6, 7], 'min_samples_leaf': [4, 8, 10],
                         'min_samples_split': [8, 10, 12]},
             scoring='accuracy')

Лучший набор значений гиперпараметров

In [113]:
best_hyperparams = grid_CV_1.best_params_
print('Лучшие значения гиперпараметров:\n', best_hyperparams)

Лучшие значения гиперпараметров:
 {'max_depth': 6, 'min_samples_leaf': 4, 'min_samples_split': 8}


Значение критерия качества для лучших значений гиперпараметров 

In [114]:
best_model = grid_CV_1.best_estimator_
print('Лучшее accuracy на тестовом множестве', best_model.score(X_test,y_test))

Лучшее accuracy на тестовом множестве 0.8986595174262735


Значение качества такое же, как и в модели с ручным подбором параметров)

In [116]:
best_model = grid_CV_1.best_estimator_
print('Лучшее accuracy на обучающем множестве', best_model.score(X_train,y_train))

Лучшее accuracy на обучающем множестве 0.8985837377438843
